In [214]:
import math
import numpy as np
import pandas as pd
validator = "Twinstake_sorted_processed.csv"
path = f"./{validator}"
df_data= pd.read_csv(path)

In [215]:
df_data.iloc[68]

block_num                                                         12713491
Block_length_target                                               7.233028
Goal_Bonded                                                         0.6666
Inflation_Rate_Change                                                 0.13
Min_Inflation_Rate                                                    0.07
Max_Inflation_Rate                                                     0.2
Min_Signatures                                                      0.6666
Blocks_per_year                                                    4360000
Validator_id             cosmosvaloper1svwt2mr4x2mx0hcmty0mxsa4rmlfau4l...
timestamp                                   2022-11-04T12:36:12.035786842Z
sign_ratio                                                             1.0
atom_staked_v                                                       185752
percent_staked                                                       64.26
commission_amt           

In [216]:
total_missed = 0
indices = df_data['block_num']
for i in range(1,len(indices)):
    if int(indices[i]) != int(indices[i-1])+1:
        total_missed += 1
        # print(i)

In [217]:
print(total_missed)
print(len(df_data))

0
304


In [218]:
df_data.reset_index(inplace=True)

In [219]:
print(df_data)

     index  block_num  Block_length_target  Goal_Bonded  \
0        0   12713423             7.233028       0.6666   
1        1   12713424             7.233028       0.6666   
2        2   12713425             7.233028       0.6666   
3        3   12713426             7.233028       0.6666   
4        4   12713427             7.233028       0.6666   
..     ...        ...                  ...          ...   
299    299   12713722             7.233028       0.6666   
300    300   12713723             7.233028       0.6666   
301    301   12713724             7.233028       0.6666   
302    302   12713725             7.233028       0.6666   
303    303   12713726             7.233028       0.6666   

     Inflation_Rate_Change  Min_Inflation_Rate  Max_Inflation_Rate  \
0                     0.13                0.07                 0.2   
1                     0.13                0.07                 0.2   
2                     0.13                0.07                 0.2   
3          

In [220]:
df_output = pd.DataFrame()
df_output['block Number'] = df_data.block_num
df_output['Atom Staked'] = df_data.atom_staked_v
df_output['Total_Rewards'] = ( df_data.commission_amt/(df_data.v_commission/100)) * 1E-6
df_output['Epoch Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Realized APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in APY'] = df_output.apply(lambda _: '', axis=1)
df_output['% Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)

print(df_output.head())

   block Number  Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0      12713423       185752     201.673529                                  
1      12713424       185752     201.681604                                  
2      12713425       185752     201.689682                                  
3      12713426       185752     201.697757                                  
4      12713427       185752     201.705829                                  

  Realized APY Expected APY Absolute Difference in Rewards  \
0                                                            
1                                                            
2                                                            
3                                                            
4                                                            

  Absolute Difference in APY % Difference in Rewards  
0                                                     
1                                               

In [221]:
# Constants from Network


# Parameters to be varied by validator
for i in range(1,len(df_data)):

  Goal_Bonded                 = float(df_data.Goal_Bonded[i]) 
  Inflation_Rate_Change       = float(df_data.Inflation_Rate_Change[i]) 
  Min_Inflation_Rate          = float(df_data.Min_Inflation_Rate[i]) 
  Max_Inflation_Rate          = float(df_data.Max_Inflation_Rate[i]) 
  Block_length                = float(df_data.block_len[i]) 
  E_Block_length              = float(df_data.Block_length_target[i])
  Blocks_per_year             = float(df_data.Blocks_per_year[i]) 
  minProposerBonus            = float(df_data.min_proposer_bonus[i]) 
  maxProposerBonus            = float(df_data.max_proposer_bonus[i]) 
  Community_Tax               = float(df_data.community_tax[i]) 
    
  Min_Signatures              = float(df_data.Min_Signatures[i]) 
  Total_Fees_Per_Block        = float(df_data.total_block_fees[i]) * 1E-6
  Bonded_Ratio                = float(df_data.percent_staked[i]) /100
  stake                       = float(df_data.atom_staked_v[i]) 
  signatures_ratio            = float(df_data.sign_ratio[i]) 
  E_Inflation_Rate_BH         = float(df_data.inflation_rate[i]) 
  totalSupply                 = float(df_data.total_supply[i]) * 1E-6

  # E_current_Inflation_rate_change = (1 - Bonded_Ratio/Goal_Bonded) * Inflation_Rate_Change
  # E_Inflation_Rate_BH             = Prev_Inflation_Rate + E_current_Inflation_rate_change / Blocks_per_year

    # Check the new inflation rate is between min and max inflation rate
  if E_Inflation_Rate_BH > Max_Inflation_Rate:
    E_Inflation_Rate_BH = Max_Inflation_Rate
  elif E_Inflation_Rate_BH < Min_Inflation_Rate:
    E_Inflation_Rate_BH = Min_Inflation_Rate

  # Real_Inflation_Rate = E_Inflation_Rate_BH * Block_length/ E_Block_length

  E_validator         = stake / (totalSupply*Bonded_Ratio) * ((1+E_Inflation_Rate_BH)**(1/Blocks_per_year)-1) * totalSupply    # technically can cancel totalSupply but left for ease of understanding
  #E_validator         = stake / (totalSupply*Bonded_Ratio) * E_Inflation_Rate_BH/Blocks_per_year * totalSupply
  E_Proposer_Bonus                     = minProposerBonus + (signatures_ratio-Min_Signatures) / (1-Min_Signatures) * (maxProposerBonus-minProposerBonus)      
  E_Proposer_stake                     = totalSupply*Bonded_Ratio/175   # THIS IS POOR ESTIMATE AS VALIDATORS WITH MORE STAKE ARE SELECTED MORE FREQUENTLY, WOULD NEED ALL VALIDATOR'S DATA TO IMPROVE MODEL
  E_transaction_reward_per_atom_staked = Total_Fees_Per_Block * (1 - Community_Tax) / (totalSupply*Bonded_Ratio + E_Proposer_stake*E_Proposer_Bonus)

  E_ProposerTxFees                     = stake * (1 + E_Proposer_Bonus) * E_transaction_reward_per_atom_staked
  E_AttesterTxFees                     = stake * E_transaction_reward_per_atom_staked

  E_Attester = E_validator + E_AttesterTxFees
  E_Proposer = E_validator + E_ProposerTxFees

  P_Proposer = stake/totalSupply    # think this is true even with round robin selection
  P_attester = 1 - P_Proposer


  E_rewards = E_Proposer*P_Proposer + E_Attester*P_attester
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
  df_output['Expected Rewards'][i] = E_rewards
  df_output['Absolute Difference in Rewards'][i] = df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]
  df_output['% Difference in Rewards'][i] = (df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]) / df_output['Epoch Rewards'][i]*100

  APY                                            = E_rewards/stake*Blocks_per_year*E_Block_length/Block_length
  Realized_APY                                   = df_output['Epoch Rewards'][i]/stake*Blocks_per_year*E_Block_length/Block_length
  df_output['Expected APY'][i]                   = APY*100
  df_output['Realized APY'][i]                   = Realized_APY*100     
  df_output['Absolute Difference in Rewards'][i] = E_rewards - df_output['Epoch Rewards'][i]
  df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])

print(df_output)

     block Number  Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0        12713423       185752     201.673529                                  
1        12713424       185752     201.681604      0.008075         0.008164   
2        12713425       185752     201.689682      0.008078         0.008161   
3        12713426       185752     201.697757      0.008075         0.008158   
4        12713427       185752     201.705829      0.008072         0.008155   
..            ...          ...            ...           ...              ...   
299      12713722       185752     204.090435      0.008099         0.008162   
300      12713723       185752     204.098511      0.008076         0.008164   
301      12713724       185752     204.106589      0.008078         0.008167   
302      12713725       185752     204.114670      0.008081         0.008189   
303      12713726       185752     204.122771      0.008101         0.008173   

    Realized APY Expected APY Absolute 

/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_19076/160660359.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_19076/160660359.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Expected Rewards'][i] = E_rewards
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_19076/160660359.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [222]:
# df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/COSMOS/single-validators/Output Files/COSMOSValidationOutput-{validator}")

In [223]:
total_rewards = df_output['Epoch Rewards'][1:].sum()
expected_total = df_output['Expected Rewards'][1:].sum()

In [224]:
# expected_total+= 15*0.008164 

In [225]:
(total_rewards-expected_total)/total_rewards*100

-1.0011410803116059

In [226]:
df_output[1:][df_output['Epoch Rewards'][1:]>0.010]

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
22,12713445,185752,201.859347,0.016198,0.008159,40.023388,20.160633,-0.008039,-19.862755,-49.62787
51,12713474,185752,202.093837,0.016165,0.008175,40.281875,20.37128,-0.00799,-19.910595,-49.428173
65,12713488,185752,202.207093,0.016184,0.008172,41.424709,20.916916,-0.008012,-20.507793,-49.506184
67,12713490,185752,202.239445,0.024274,0.00818,58.330665,19.658023,-0.016093,-38.672642,-66.298991
74,12713497,185752,202.279888,0.016169,0.008166,42.350225,21.387623,-0.008004,-20.962601,-49.498206
79,12713502,185752,202.320304,0.016184,0.008159,43.093657,21.725957,-0.008025,-21.367701,-49.584328
102,12713525,185752,202.506233,0.016202,0.008168,40.506152,20.42022,-0.008034,-20.085932,-49.587361
111,12713534,185752,202.578981,0.016156,0.008168,42.905298,21.69084,-0.007988,-21.214458,-49.444846
113,12713536,185752,202.595148,0.016167,0.008167,42.3042,21.370495,-0.008,-20.933705,-49.483751
116,12713539,185752,202.619399,0.016173,0.008187,43.371006,21.955185,-0.007986,-21.415821,-49.378198
